# Weekly Report for Hospital Data

In [2]:
# Imports
import sys
import pandas as pd
import psycopg
import matplotlib.pyplot as plt

from credentials import DB_PASSWORD, DB_USER

In [3]:
# Connect to DB
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname=DB_USER,
    user=DB_USER, password=DB_PASSWORD
)
cur = conn.cursor()

In [4]:
# Summary I & II
with conn.transaction():
    cur.execute(
        "SELECT report_date AS week, COUNT(*) AS records_loaded, "
        "SUM(total_adult_hospital_beds), SUM(total_pediatric_hospital_beds), "
        "SUM(total_adult_hospital_beds_occupied), "
        "SUM(total_pediatric_hospital_beds_occupied), "
        "SUM(inpatient_beds_occupied_covid) "
        "FROM facility_reports "
        "GROUP BY report_date "
        "ORDER BY week DESC;"
    )

records = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Records Loaded", "Total Adult Hospital Beds Available", 
                             "Total Pediatric Hospital Beds Available", 
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total Hospital Beds Used by COVID cases"]
)

### Record loaded-in each week

In [5]:
# I
records[["Week", "Records Loaded"]]

,Week,Records Loaded
0,2022-10-21,4985
1,2022-10-14,4995
2,2022-10-07,4996
3,2022-09-30,4997
4,2022-09-23,4996


### Hospital beds information each week

In [6]:
# II
records

,Week,Records Loaded,Total Adult Hospital Beds Available,Total Pediatric Hospital Beds Available,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total Hospital Beds Used by COVID cases
0,2022-10-21,4985,414880.3,39882.2,469153.4,29706.5,24116.8
1,2022-10-14,4995,412939.0,39711.6,467145.9,28799.0,23695.5
2,2022-10-07,4996,415465.4,39796.8,465980.5,28285.8,23689.3
3,2022-09-30,4997,416279.4,39804.7,462204.5,28042.8,24142.5
4,2022-09-23,4996,413589.5,39605.5,463698.4,27778.1,25600.6


In [27]:
# III
with conn.transaction():
    cur.execute(
        "SELECT rating, "
        "ROUND(SUM(total_adult_hospital_beds_occupied)/SUM(total_adult_hospital_beds)*100, 2), "
        "ROUND(SUM(total_pediatric_hospital_beds_occupied)/SUM(total_pediatric_hospital_beds)*100, 2), "
        "ROUND(SUM(total_icu_beds_occupied)/SUM(total_icu_beds)*100, 2) "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "WHERE rating IS NOT NULL "
        "AND total_adult_hospital_beds_occupied < total_adult_hospital_beds "
        "AND report_date = %(week)s"
        "GROUP BY rating "
        "ORDER BY rating DESC;",
        {"week": pd.to_datetime("2022-10-21")}
    )

ratings = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Rating", "Percent % Adult Hospital Beds In-Use", 
                             "Percent % Pediatric Hospital Beds In-Use",
                             "Percent % ICU Beds In-Use"]
)

### Percent Hospital Beds In-Use by Hospital Rating

In [28]:
ratings

,Hospital Rating,Percent % Adult Hospital Beds In-Use,Percent % Pediatric Hospital Beds In-Use,Percent % ICU Beds In-Use
0,5,66.36,65.74,67.25
1,4,68.24,69.45,71.42
2,3,68.60,64.14,69.24
3,2,66.66,68.47,70.86
4,1,62.95,59.01,72.28


In [71]:
# No data
# Note: Change thisweek to the parameter of this week
with conn.transaction():
    cur.execute(
        "SELECT facility_name, MAX(report_date) "
        "FROM facility_information "
        "JOIN facility_reports ON "
        "facility_id = hospital_pk "
        "GROUP BY facility_id "
        "HAVING MAX(report_date) < %(week)s "
        "ORDER BY MAX(report_date) DESC;",
        {"week": pd.to_datetime("2022-10-21")}
    )

nodata = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Last reported Date"]
)

In [20]:
# IV
with conn.transaction():
    # Extract the number of beds in use    
    cur.execute("SELECT report_date, SUM(total_occupied_0) "
                "AS total_occupied_all, "
                "SUM(inpatient_beds_occupied_covid) "
                "AS total_occupied_covid FROM "
                "(SELECT report_date, inpatient_beds_occupied_covid, "
                "total_adult_hospital_beds_occupied + "
                "total_pediatric_hospital_beds_occupied + "
                "total_icu_beds_occupied + "
                "inpatient_beds_occupied_covid AS total_occupied_0 "
                "FROM facility_reports) AS a "
                "GROUP BY report_date "
                "ORDER BY report_date;")

data_used = pd.DataFrame(cur.fetchall(), 
                         columns=["report_date", 
                                  "all_used_beds",
                                  "covid_used_beds"]
)

OperationalError: consuming input failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

In [19]:
# Make the bar plot
plt.figure(figsize=(7,4))
labels = data_used["report_date"]
x = np.arange(len(labels))
width = 0.25
plt.bar(x-width/2, data_used["all_used_beds"], width, 
        label = "All cases", tick_label=labels, fc="aquamarine")
plt.bar(x+width/2, data_used["covid_used_beds"], width, 
        label = "Covid cases", fc="lightseagreen")
#plt.ylim(0, 550000)
plt.xlabel("Report date")
plt.ylabel("The number of occupied beds")
plt.legend(loc="best")
plt.title("The number of beds in use of all cases and covid cases")
# save the plot, but we might not need it in output

# Make the pie chart
fig,axes1 = plt.subplots(1,3)
plt.show()

NameError: name 'data_used' is not defined

<Figure size 504x288 with 0 Axes>

In [72]:
nodata

,Hospital Name,Last reported Date
0,WAGONER COMMUNITY HOSPITAL,2022-10-14
1,DRUMRIGHT REGIONAL HOSPITAL,2022-10-14
2,KNOX COUNTY HOSPITAL,2022-10-14
3,SYRACUSE AREA HEALTH,2022-10-14
4,UNITY MEDICAL CENTER,2022-10-14
5,WINSTON MEDICAL CENTER & SWINGBED,2022-10-14
6,MASSAC MEMORIAL HOSPITAL,2022-10-14
7,HOLY CROSS HOSPITAL,2022-10-14
8,LA PAZ REGIONAL HOSPITAL,2022-10-14
9,MT SINAI HOSPITAL MEDICAL CENTER,2022-10-14


In [104]:
# State COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT tw.state, tw.covidcase - lw.covidcase AS coviddiff "
        "FROM ("
        # This weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        "GROUP BY state) tw "
        "JOIN ("
        # Last weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        "GROUP BY state) lw "
        "ON tw.state = lw.state "
        "ORDER BY coviddiff DESC;",
        {
            "week": pd.to_datetime("2022-10-21"),
            "lastweek": pd.to_datetime("2022-10-21") - pd.Timedelta(days=7)
        }
    )

statecovid = pd.DataFrame(
    cur.fetchall(), columns=["States", "Change in Covid Case"]
)

In [105]:
statecovid

,States,Change in Covid Case
0,GU,None
1,VI,None
2,IL,133.3
3,WI,81.4
4,OH,79.9
5,IN,67.8
6,MN,65.9
7,AZ,64.3
8,PA,60.4
9,MO,50.1


In [110]:
# Top Hospital COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT info.facility_name, tw.covidcase - lw.covidcase AS coviddiff, "
        "address, city, state, zipcode "
        "FROM ("
        # This weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        ") tw "
        "JOIN ("
        # Last weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        ") lw "
        "ON tw.facility_id = lw.facility_id "
        "JOIN facility_information info "
        "ON info.facility_id = tw.facility_id "
        "WHERE tw.covidcase - lw.covidcase IS NOT NULL "
        "ORDER BY coviddiff DESC "
        "LIMIT 5",
        {
            "week": pd.to_datetime("2022-10-21"),
            "lastweek": pd.to_datetime("2022-10-21") - pd.Timedelta(days=7)
        }
    )

hospitalcovid = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Change in Covid Case",
                             "Address", "City", "State", "Zipcode"]
)

In [111]:
hospitalcovid

,Hospital Name,Change in Covid Case,Address,City,State,Zipcode
0,CAROLINAS MEDICAL CENTER/BEHAV HEALTH,29.4,1000 BLYTHE BLVD,CHARLOTTE,NC,28203
1,NEW YORK-PRESBYTERIAN HOSPITAL,24.7,525 EAST 68TH STREET,NEW YORK,NY,10065
2,GEISINGER MEDICAL CENTER,17.7,100 NORTH ACADEMY AVENUE,DANVILLE,PA,17822
3,HOUSTON METHODIST HOSPITAL,16.2,6565 FANNIN,HOUSTON,TX,77030
4,MAINE MEDICAL CENTER,15.8,22 BRAMHALL ST,PORTLAND,ME,4102


In [7]:
with conn.transaction():
    cur.execute(
        "SELECT report_date, SUM(inpatient_beds_occupied_covid) "
        "FROM facility_information JOIN "
        "facility_reports ON "
        "facility_id = hospital_pk "
        "WHERE state = 'GU' "
        "GROUP BY report_date;"
    )

checkstate = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Covid Case"]
)

checkstate

,Week,Covid Case
0,2022-09-23,5.6
1,2022-09-30,5
2,2022-10-07,None
3,2022-10-14,None
4,2022-10-21,None
